In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import mplfinance as mpf
from mplfinance import plot as mplot
import os
from tensorflow import keras
from matplotlib.image import imread
from datetime import date
import datetime
from datetime import datetime as dt
import matplotlib 
matplotlib.use('agg')
import matplotlib.pyplot as plt
import re
import tensorflow
import cv2 as cv
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import gc
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_position_sell(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + (sl * price),
        "tp": price - (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [3]:
def get_position_buy(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - (sl * price),
        "tp": price + (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [4]:
# currency_pairs= ['AUDCAD', 'AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY',
#                  'EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD',
#                  'GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPNZD','GBPUSD','NZDJPY',
#                  'NZDCAD','NZDCHF','NZDUSD','USDCAD','USDCHF','USDJPY']

In [5]:
currency_pairs= ['AUDCAD', 'AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY',
                 'EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD',
                 'GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPNZD','GBPUSD','NZDJPY',
                 'NZDCAD','NZDCHF','NZDUSD','USDCAD','USDCHF','USDJPY']

In [6]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

mt5.account_info()

AccountInfo(login=5386640, trade_mode=0, leverage=500, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=12809.41, credit=0.0, profit=-681.39, equity=12128.02, margin=1111.08, margin_free=11016.94, margin_level=1091.5523634661772, margin_so_call=50.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Zahed Zahedi', server='AMarkets-Demo', currency='USD', company='AMarkets')

In [7]:
image_path_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h'
image_path_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h'
RecolPath_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h\Recolored'
RecolPath_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h\Recolored'
model_1h= tensorflow.keras.models.load_model('ckpt_03_CorrectHighLowWithoutResize.hdf5')
model_4h_old= tensorflow.keras.models.load_model('ckpt_04_4h_best.hdf5')
model_4h_new= tensorflow.keras.models.load_model('ckpt_04_tp008sl003_best_4h.hdf5')
df_minmax_1h= pd.read_csv('df_minmax_mrged_1h.csv')
df_minmax_4h= pd.read_csv('df_minmax_merged_4h.csv')
tesh_optimum_1h = pd.read_csv('df_best_final_EmbedRecolor_withoutResize_CorrectHL.csv')#tresh 5 candle
tesh_optimum_4h = pd.read_excel('Best_TreshFor_4h_Merged.xlsx')#current
tesh_optimum_4h_old = pd.read_excel('Best_TreshFor_4h_Merged_old.xlsx')#tresh 4h
# Selected_pairs_for_Old_model_4h= ['CHFJPYb', 'AUDUSDb', 'GBPCADb', 'EURJPYb', 'GBPJPYb', 'EURNZDb', 'EURGBPb',
#                                   'NZDUSDb', 'AUDCHFb', 'AUDCADb', 'NZDCHFb', 'EURCADb', 'EURUSDb', 'GBPUSDb', 'NZDCADb']
# Selected_pairs_for_New_model_4h= ['USDCADb', 'EURAUDb', 'EURCHFb', 'GBPAUDb', 'CADJPYb', 'AUDNZDb',
#                                   'USDJPYb', 'CADCHFb', 'GBPCHFb', 'AUDJPYb', 'GBPNZDb', 'NZDJPYb', 'USDCHFb']

In [8]:
Selected_pairs_for_Old_model_4h= ['CHFJPY', 'AUDUSD', 'GBPCAD', 'EURJPY', 'GBPJPY', 'EURNZD', 'EURGBP',
                                  'NZDUSD', 'AUDCHF', 'AUDCAD', 'NZDCHF', 'EURCAD', 'EURUSD', 'GBPUSD', 'NZDCAD']
Selected_pairs_for_New_model_4h= ['USDCAD', 'EURAUD', 'EURCHF', 'GBPAUD', 'CADJPY', 'AUDNZD',
                                  'USDJPY', 'CADCHF', 'GBPCHF', 'AUDJPY', 'GBPNZD', 'NZDJPY', 'USDCHF']

In [9]:
mc = mpf.make_marketcolors(up='g',down='r')
s  = mpf.make_mpf_style(marketcolors=mc)

In [10]:
def make_candle_stick(df, image_path, pair):
    df['time']= df.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
    df['time']= pd.to_datetime(df['time'])
    time_log= df.iloc[48]['time']
    
    openn = df.iloc[48]['open']
    high = df.iloc[48]['high']
    low = df.iloc[48]['low']
    close = df.iloc[48]['close']
    df = df.set_index('time')
    df.rename(columns={'tick_volume':'volume'}, inplace=True)
    
    HighLow= str(np.max(df.iloc[1:, 1])/np.min(df.iloc[1:, 2])).replace('.', '')[:6]
    if '_' in HighLow:
        HighLow = HighLow.replace('_', '0')
    
    #Heiken Ashi
    df_temp_Ha = df.copy()
    for i in range(1, len(df_temp_Ha)):
        df_temp_Ha.iloc[i, 0] = (df_temp_Ha.iloc[i-1, 0] + df_temp_Ha.iloc[i-1, 3])/2
        df_temp_Ha.iloc[i, 3] = (df_temp_Ha.iloc[i, 0] + df_temp_Ha.iloc[i, 1] + df_temp_Ha.iloc[i, 2] + df_temp_Ha.iloc[i, 3])/4
    
    time_log = str(time_log).replace(':', '').replace(' ', '').replace('-', '')
    
    img_name = "".join([pair, '_' ,str(time_log),'_', HighLow, '.jpg'])
    img_path= os.path.join(image_path, img_name)
    
    mplot(df_temp_Ha.iloc[1:-1, :], type='candle', volume=False, figscale=2,figratio=(10,10), style=s, savefig=img_path)
    return time_log, img_path, openn, high, low, close


In [11]:
# font = ImageFont.truetype('times.ttf', 20)
font = cv.FONT_HERSHEY_COMPLEX
loc= (600, 795)
font_scale= 1.1
color = (0, 0, 0)
thickness= 3

# print(df_minmax)

In [12]:
def makeEmbedImage(img_name, RecolPath, df_minmax, pair):
    img = cv.imread(img_name)
    img = img[140:-207, 220: -120, :]
    hl = int(img_name.split('.')[0].split('_')[-1])
    if hl < 100:
        hl = hl * 10000
    elif hl < 1000:
        hl = hl * 1000
    elif hl < 10000:
        hl = hl * 100
    elif hl < 100000:
        hl = hl * 10

    mx = df_minmax.loc[0, pair]
    mn = df_minmax.loc[1, pair]
    hl = (hl - mn) / (mx - mn)
    w = np.uint8(((hl * 255)))
    img[771:850, 20:550, 2] = w
    img[771:850, 20:550, 0] = w
    img[771:850, 20:550, 1] = 0
    img = cv.putText(img, pair, loc, font, font_scale, color, thickness, cv.LINE_AA)
    img = cv.resize(img, (300, 300))
    name= img_name.split('\\')[-1]
    cv.imwrite(os.path.join(RecolPath, name), img)
    return img


In [13]:
str(dt.now()), str(dt.now()).split(':')[0].split(' ')[-1]

('2022-01-24 18:09:41.711405', '18')

In [14]:
# tp= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','t'].values[0]
# sl= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','s'].values[0]
# acc_u= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','u'].values[0]
# acc_d= tesh_optimum_4h.loc[tesh_optimum_4h['p']== 'EURUSD','d'].values[0]
# tp, sl, acc_u, acc_d

In [15]:
def strategy_h1_24(pair,acc_1h ):
    acc_u= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'u'].values[0]
    acc_d= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'d'].values[0]
    if acc_1h >= acc_u:
        print('------ Buy Position_h1_24 -------')
        position=get_position_buy(pair, 0.01, 0.009 , 0.002)
        strategy_h1_24_positions[position.order]={"pair":pair, "hour":1}
    elif acc_1h <= acc_d:
        print('--------- Sell Position_h1_24 --------')
        position=get_position_sell(pair, 0.01, 0.009 , 0.002)
        strategy_h1_24_positions[position.order]={"pair":pair, "hour":1}

In [16]:
def strategy_h1_5(pair,acc_1h):
    acc_u= tesh_optimum_1h.loc[tesh_optimum_1h['p']== pair,'u'].values[0]
    acc_d= tesh_optimum_1h.loc[tesh_optimum_1h['p']== pair,'d'].values[0]
    if acc_1h >= acc_u:
        print('------ Buy Position_h1_5 -------')
        position=get_position_buy(pair, 0.02, 0.009 , 0.002)
        strategy_h1_5_positions[position.order]={"pair":pair, "hour":1}
    elif acc_1h <= acc_d:
        print('--------- Sell Position_h1_5 --------')
        position=get_position_sell(pair, 0.02, 0.009 , 0.002)
        strategy_h1_5_positions[position.order]={"pair":pair, "hour":1}

In [17]:
def strategy_h4_5(pair,acc_4h):
    acc_u= tesh_optimum_4h_old.loc[tesh_optimum_4h_old['p']== pair,'u'].values[0]
    acc_d= tesh_optimum_4h_old.loc[tesh_optimum_4h_old['p']== pair,'d'].values[0]
    if acc_4h >= acc_u:
        print('------ Buy Position_h4_5 -------')
        position=get_position_buy(pair, 0.03, 0.009 , 0.002)
        strategy_h4_5_positions[position.order]={"pair":pair, "hour":1}
    elif acc_4h <= acc_d:
        print('--------- Sell Position_h4_5 --------')
        position=get_position_sell(pair, 0.03, 0.009 , 0.002)
        strategy_h4_5_positions[position.order]={"pair":pair, "hour":1}

In [18]:
def startegy_h1_24_close():
    for position in list(strategy_h1_24_positions.keys()):
        strategy_h1_24_positions[position]["hour"]+=1
        if strategy_h1_24_positions[position]["hour"]==24:
            try:
                mt5.Close(strategy_h1_24_positions[position]["pair"],ticket=position)
            except:
                pass
            strategy_h1_24_positions.pop(position, None)

In [19]:
def startegy_h1_5_close():
    for position in list(strategy_h1_5_positions.keys()):
        strategy_h1_5_positions[position]["hour"]+=1
        if strategy_h1_5_positions[position]["hour"]==6:
            try:
                mt5.Close(strategy_h1_5_positions[position]["pair"],ticket=position)
            except:
                pass
            strategy_h1_5_positions.pop(position, None)

In [20]:
def startegy_h4_5_close():
    for position in list(strategy_h4_5_positions.keys()):
        strategy_h4_5_positions[position]["hour"]+=1
        if strategy_h4_5_positions[position]["hour"]==6:
            try:
                mt5.Close(strategy_h4_5_positions[position]["pair"],ticket=position)
            except:
                pass
            strategy_h4_5_positions.pop(position, None)

In [21]:
def make_predict_h1():
    list_log_dashboard= []
    list_log = []
    for pair in currency_pairs:
#         tp= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'t'].values[0]
#         sl= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'s'].values[0]
        

        rates_1h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, 50)
        df_1h=pd.DataFrame(rates_1h).reset_index(drop=True)
        time_log, img_name_1h, open_1h, high_1h, low_1h, close_1h = make_candle_stick(df_1h, image_path_1h, pair)
        img_1h= makeEmbedImage(img_name_1h, RecolPath_1h, df_minmax_1h, pair)
        acc_1h=model_1h.predict(np.array([img_1h,img_1h]))[0,1]

        rates_4h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H4, 0, 50)
        df_4h=pd.DataFrame(rates_4h).reset_index(drop=True)
        time_log, img_name_4h, open_4h, high_4h, low_4h, close_4h = make_candle_stick(df_4h, image_path_4h, pair)
        img_4h= makeEmbedImage(img_name_4h, RecolPath_4h, df_minmax_4h, pair)
        if pair in Selected_pairs_for_Old_model_4h:
            acc_4h=model_4h_old.predict(np.array([img_4h,img_4h]))[0,1]
        else:
            acc_4h=model_4h_new.predict(np.array([img_4h,img_4h]))[0,1]
        print(f'{pair}, acc_1h:{acc_1h}, acc_4h:{acc_4h}')

        strategy_h1_24(pair,acc_1h)
        strategy_h1_5(pair,acc_1h)
        

        list_log.append([time_log, pair, img_name_1h, img_name_4h, acc_1h, acc_4h,open_1h, high_1h, low_1h, close_1h, open_4h, high_4h, low_4h, close_4h])
        list_log_dashboard.append([pair, acc_1h, acc_4h])

        if pair == currency_pairs[-1]:
            arr_log_dashboard = np.array(list_log_dashboard)
            arr_log_dashboard= arr_log_dashboard.reshape(int(len(currency_pairs)), 3)
            df_dashboard= pd.DataFrame(arr_log_dashboard, columns=['PAIR','1H', '4H'])
            df_dashboard.to_excel(r'C:\xampp\htdocs\fx_dashboard\pred.xlsx')



            arr_log = np.array(list_log)
            arr_log= arr_log.reshape(int(len(currency_pairs)), 14)
            df_temp = pd.DataFrame(arr_log, columns=['time_log','pair', 'img_name_1h','img_name_4h', 'acc_1h',
                                                     'acc_4h','open_1h', 'high_1h',  'low_1h','close_1h',
                                                    'open_4h','high_4h', 'low_4h', 'close_4h'])
    return df_temp

In [22]:
def make_predict_h1_h4():
    list_log_dashboard= []
    list_log = []
    for pair in currency_pairs:
#         tp= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'t'].values[0]
#         sl= tesh_optimum_4h.loc[tesh_optimum_4h['p']== pair,'s'].values[0]
        

        rates_1h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, 50)
        df_1h=pd.DataFrame(rates_1h).reset_index(drop=True)
        time_log, img_name_1h, open_1h, high_1h, low_1h, close_1h = make_candle_stick(df_1h, image_path_1h, pair)
        img_1h= makeEmbedImage(img_name_1h, RecolPath_1h, df_minmax_1h, pair)
        acc_1h=model_1h.predict(np.array([img_1h,img_1h]))[0,1]

        rates_4h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H4, 0, 50)
        df_4h=pd.DataFrame(rates_4h).reset_index(drop=True)
        time_log, img_name_4h, open_4h, high_4h, low_4h, close_4h = make_candle_stick(df_4h, image_path_4h, pair)
        img_4h= makeEmbedImage(img_name_4h, RecolPath_4h, df_minmax_4h, pair)
        if pair in Selected_pairs_for_Old_model_4h:
            acc_4h=model_4h_old.predict(np.array([img_4h,img_4h]))[0,1]
        else:
            acc_4h=model_4h_new.predict(np.array([img_4h,img_4h]))[0,1]
        print(f'{pair}, acc_1h:{acc_1h}, acc_4h:{acc_4h}')

        strategy_h1_24(pair,acc_1h)
        strategy_h1_5(pair,acc_1h)
        strategy_h4_5(pair,acc_4h)

        list_log.append([time_log, pair, img_name_1h, img_name_4h, acc_1h, acc_4h,open_1h, high_1h, low_1h, close_1h, open_4h, high_4h, low_4h, close_4h])
        list_log_dashboard.append([pair, acc_1h, acc_4h])

        if pair == currency_pairs[-1]:
            arr_log_dashboard = np.array(list_log_dashboard)
            arr_log_dashboard= arr_log_dashboard.reshape(int(len(currency_pairs)), 3)
            df_dashboard= pd.DataFrame(arr_log_dashboard, columns=['PAIR','1H', '4H'])
            df_dashboard.to_excel(r'C:\xampp\htdocs\fx_dashboard\pred.xlsx')



            arr_log = np.array(list_log)
            arr_log= arr_log.reshape(int(len(currency_pairs)), 14)
            df_temp = pd.DataFrame(arr_log, columns=['time_log','pair', 'img_name_1h','img_name_4h', 'acc_1h',
                                                     'acc_4h','open_1h', 'high_1h',  'low_1h','close_1h',
                                                    'open_4h','high_4h', 'low_4h', 'close_4h'])
    return df_temp

In [ ]:
strategy_h1_24_positions={}
strategy_h1_5_positions={}
strategy_h4_5_positions={}
h_list= ['00', '04', '08', '12', '16', '20']
i=0
while(True):
    time = dt.now()
    min_part= str(time).split(':')[1]
    sec_part= str(time).split(':')[-1].split('.')[0]
    h_part= str(time).split(':')[0].split(' ')[-1]
    if (h_part in h_list) & (min_part == '59') & (int(sec_part) >= 55):#check 4 hour
        startegy_h1_24_close()
        startegy_h1_5_close()
        startegy_h4_5_close()
        print(f'---- {time} ------ ')
        try:
            df_temp= make_predict_h1_h4()
            file_name= r'C:\Users\Administrator\Desktop\Trade_1005\Logs_1005/df_log_{0}_6.csv'.format(str(i))
            if i == 0:
                df_temp.to_csv('df_log.csv', index= False)
                df_temp.to_csv(file_name, index= False)
            else:
                df_log= pd.read_csv('df_log.csv')
                df_log = pd.concat([df_log, df_temp], axis = 0)
                df_log.to_csv('df_log.csv', index= False)
                df_log.to_csv(file_name, index= False)
            i+=1
        except Exception as e:
            print(e)
    
    
    elif (min_part == '59') & (int(sec_part) >= 55):
        startegy_h1_24_close()
        startegy_h1_5_close()
        print(f'---- {time} ------ ')
        try:
            df_temp= make_predict_h1()
            file_name= r'C:\Users\Administrator\Desktop\Trade_1005\Logs_1005/df_log_{0}_6.csv'.format(str(i))
            if i == 0:
                df_temp.to_csv('df_log.csv', index= False)
                df_temp.to_csv(file_name, index= False)
            else:
                df_log= pd.read_csv('df_log.csv')
                df_log = pd.concat([df_log, df_temp], axis = 0)
                df_log.to_csv('df_log.csv', index= False)
                df_log.to_csv(file_name, index= False)
            i+=1
        except Exception as e:
            print(e)
            

---- 2022-01-24 18:59:55.007214 ------ 
AUDCAD, acc_1h:0.5297167301177979, acc_4h:0.29629048705101013
AUDCHF, acc_1h:0.23420369625091553, acc_4h:0.3758557438850403
AUDJPY, acc_1h:0.17802734673023224, acc_4h:0.743308424949646
AUDNZD, acc_1h:0.38562649488449097, acc_4h:0.49338847398757935
AUDUSD, acc_1h:0.1593586951494217, acc_4h:0.12431351840496063
CADCHF, acc_1h:0.20100045204162598, acc_4h:0.7949114441871643
CADJPY, acc_1h:0.04420749470591545, acc_4h:0.7077257037162781
CHFJPY, acc_1h:0.2936680316925049, acc_4h:0.6625447869300842
EURAUD, acc_1h:0.6894853115081787, acc_4h:0.052018918097019196
EURCAD, acc_1h:0.42334654927253723, acc_4h:0.04209088906645775
EURCHF, acc_1h:0.2181280255317688, acc_4h:0.7839618921279907
EURGBP, acc_1h:0.4907066524028778, acc_4h:0.16908100247383118
EURJPY, acc_1h:0.307941198348999, acc_4h:0.4946112036705017
EURNZD, acc_1h:0.5643423199653625, acc_4h:0.2383987456560135
EURUSD, acc_1h:0.38839131593704224, acc_4h:0.3698177635669708
GBPAUD, acc_1h:0.3309116363525390

GBPCHF, acc_1h:0.08386370539665222, acc_4h:0.05266927555203438
GBPJPY, acc_1h:0.026000749319791794, acc_4h:0.1583997905254364
GBPNZD, acc_1h:0.3205000162124634, acc_4h:0.45118042826652527
GBPUSD, acc_1h:0.14956358075141907, acc_4h:0.2903115749359131
NZDJPY, acc_1h:0.04639587178826332, acc_4h:0.32660332322120667
NZDCAD, acc_1h:0.3907744288444519, acc_4h:0.43300628662109375
NZDCHF, acc_1h:0.2978135049343109, acc_4h:0.17364990711212158
NZDUSD, acc_1h:0.38284358382225037, acc_4h:0.23255999386310577
USDCAD, acc_1h:0.5290736556053162, acc_4h:0.23625585436820984
USDCHF, acc_1h:0.09445490688085556, acc_4h:0.8003208041191101
USDJPY, acc_1h:0.1603117138147354, acc_4h:0.40417587757110596
---- 2022-01-24 23:59:55.007338 ------ 
AUDCAD, acc_1h:0.25387507677078247, acc_4h:0.02361583523452282
AUDCHF, acc_1h:0.006808511912822723, acc_4h:0.014954711310565472
AUDJPY, acc_1h:0.022164424881339073, acc_4h:0.6887997388839722
AUDNZD, acc_1h:0.06610679626464844, acc_4h:0.6636651754379272
AUDUSD, acc_1h:0.1634

USDJPY, acc_1h:0.21716250479221344, acc_4h:0.4594031572341919
---- 2022-01-25 03:59:55.014211 ------ 
AUDCAD, acc_1h:0.20326067507266998, acc_4h:0.04813172295689583
AUDCHF, acc_1h:0.11197621375322342, acc_4h:0.0010533669264987111
AUDJPY, acc_1h:0.05665115267038345, acc_4h:0.21102091670036316
AUDNZD, acc_1h:0.050590816885232925, acc_4h:0.7092817425727844
AUDUSD, acc_1h:0.0457923524081707, acc_4h:0.030801361426711082
CADCHF, acc_1h:0.11749540269374847, acc_4h:0.18106338381767273
CADJPY, acc_1h:0.08784219622612, acc_4h:0.07686885446310043
CHFJPY, acc_1h:0.00360679579898715, acc_4h:0.3125872015953064
--------- Sell Position_h1_24 --------
124.46200900000001
EURAUD, acc_1h:0.3536894619464874, acc_4h:0.9915316700935364
EURCAD, acc_1h:0.5513292551040649, acc_4h:0.7069031596183777
EURCHF, acc_1h:0.16879770159721375, acc_4h:0.671561598777771
EURGBP, acc_1h:0.4945850670337677, acc_4h:0.8613319993019104
EURJPY, acc_1h:0.15912894904613495, acc_4h:0.10469567775726318
EURNZD, acc_1h:0.48590272665023

EURJPY, acc_1h:0.4033162593841553, acc_4h:0.030232757329940796
EURNZD, acc_1h:0.7123560905456543, acc_4h:0.921927809715271
EURUSD, acc_1h:0.8271967172622681, acc_4h:0.021974097937345505
GBPAUD, acc_1h:0.16707900166511536, acc_4h:0.19060486555099487
GBPCAD, acc_1h:0.45687609910964966, acc_4h:0.6928257942199707
GBPCHF, acc_1h:0.261769562959671, acc_4h:0.4171418845653534
GBPJPY, acc_1h:0.026449069380760193, acc_4h:0.04810931533575058
GBPNZD, acc_1h:0.32924777269363403, acc_4h:0.6409493684768677
GBPUSD, acc_1h:0.05370859429240227, acc_4h:0.0046895574778318405
NZDJPY, acc_1h:0.05502229556441307, acc_4h:0.7815051078796387
NZDCAD, acc_1h:0.12584377825260162, acc_4h:0.4009217619895935
NZDCHF, acc_1h:0.1867169439792633, acc_4h:0.08585591614246368
NZDUSD, acc_1h:0.0589660108089447, acc_4h:0.07503197342157364
USDCAD, acc_1h:0.8762783408164978, acc_4h:0.9189210534095764
USDCHF, acc_1h:0.03765216842293739, acc_4h:0.4378865659236908
USDJPY, acc_1h:0.22871029376983643, acc_4h:0.16100789606571198
